In [1]:
#!/usr/bin/env python
import SciServer.LoginPortal as Login
token=Login.getToken()
user = Login.getKeystoneUserWithToken(token)
import SciServer.CasJobs as CasJobs # query with CasJobs
import SciServer.SciDrive           # read/write to/from SciDrive
import urllib
from bs4 import BeautifulSoup as bs
import pandas
import numpy as np
import os
import datetime
import dateutil
print(token)

424193872ec546a8a8c4a32b4e7b5018


In [2]:
# First, get the list of dead celebrity names from the CSV file in SciDrive
#def scidrivePublicURL(path):
#    req = urllib.request.Request(url=SciServer.Config.SciDriveHost+'/vospace-2.0/1/media/sandbox/'+path,method='GET') 
#    req.add_header('X-Auth-Token', token) 
#    req.add_header('Content-Type','application/xml') 
#    res=urllib.request.urlopen(req)
#    jsonResponse = json.loads(res.read().decode())
#    return jsonResponse['url']
#container = 'fame'
#filename = container+"/death1732.csv"

# Load the list of names, first into a pandas dataframe, then into an Nx1 numpy array.
#rawdata = SciServer.SciDrive.download(filename,token=token)
#deadpandas = pandas.read_csv('deathunicode2.csv',encoding='utf-8')
deadpandas = pandas.read_csv('deathin5.csv')

deadpeople = deadpandas.values
#print(deadpeople)

# The article_lengths array (also Nx1 numpy) will hold length of each Wikip article in chars
article_lengths = np.zeros([len(deadpeople),1],dtype=np.int) 

for i in range(0,len(deadpeople)): # iterate through each name
#    print(i)
    # Construct the Wikip URL and read its contents
    name = str(deadpeople[i,0])
#    print(name)
    name2 = urllib.parse.quote(name.replace(" ","_"))
#    print(name2)
    url = "http://en.wikipedia.org/wiki/"+name2
#    print(i,":",url)
    page = urllib.request.urlopen(url).read()
    # Parse HTML to find the contents of every <p>...</p> tag
    soup = bs(page, "html.parser")
#    soup.prettify()
#    heading1 = soup.find('h1')
    x = soup.find_all('p')
    
    # Calculate the length of all the text within the <p>...</p> tags
    lent = 0
    for j in x:
        lent = lent + len(j.get_text())
    article_lengths[[i][0]] = lent
    
    # instant gratification: print the loop variable every 50 turns so I see it's doing something
    if (i % 50 == 0):
        print(i)
     #   print(article_lengths[i][0])

# Add article length to list of names
deadpeople = np.hstack((deadpeople,article_lengths))

print('Done!')
print(deadpeople)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
Done!
[['S. R. Nathan' 13133]
 ['Sabam Siagian' 513]
 ['Sabri Yirmibeşoğlu' 2795]
 ..., 
 ['Zygmunt Kukla' 314]
 ['Zygmunt Zimowski' 5807]
 ['Zyta Gilowska' 1522]]


In [3]:
# Results will eventually be in "deadpeople" - come back to it at the end

# Store the number of revisions & editors in a new array called "revisionarray"
revisionarray = np.zeros([len(deadpeople),2],dtype=int)

#revisionarray = np.zeros([1,2],dtype=int)
#for i in range(0,1):

# Iterate through all the names in deadpeople
for i in range(0,len(deadpeople)):
    name = str(deadpeople[i,0])
#    name = "David Bowie"
#    name = "Agha Nasir"
#    name = "Marvin Minsky"
    name2 = urllib.parse.quote(name.replace(" ","_"))
    print(name)
    
    # Wikipedia displays revisions max 500 at a time. 
    iteration = 0
    completed = 0
    earliestrev = ''
    rownumber = 0
    
    while (completed == 0):
        iteration = iteration + 1
#        print('top of while loop.')
#        print("iteration: ",iteration)
#        print(rownumber)
#        print('earliestrev = ',earliestrev)
        historyurl = "https://en.wikipedia.org/w/index.php?title="
        historyurl += name2
        historyurl += "&offset="
        historyurl += earliestrev
        historyurl += "&limit=500&action=history"
#        print("historyurl = ",historyurl)

        page = urllib.request.urlopen(historyurl).read()
        soup = bs(page, "html.parser")

        histolist = soup.find('ul',id='pagehistory')
        revisions = histolist.find_all('li')
#        print("length of revisions = ",len(revisions))        
        
        if (iteration == 1):
            revisor = np.empty([len(revisions),4],dtype='object')
#            print('revisor initialized to length ',len(revisor))
        else:
            moreroom = np.empty(([len(revisions),4]),dtype='object')
            revisor = np.vstack((revisor,moreroom))
#        print("size of revisor = ",revisor.shape)

        if (len(revisions) < 500):
            completed = 1
#        print("lastpage = ",lastpage)
        for j in revisions:
#            print(name)
#            print(rownumber)
#            print(j)
            if j.find(class_='history-deleted'):
                revisor[rownumber] = [dateutil.parser.parse('9999-12-31 23:59'),'','','']
            else:
    #            print(dateutil.parser.parse(j.find(class_='mw-changeslist-date').get_text()))
                revisor[rownumber,0] = dateutil.parser.parse(j.find(class_='mw-changeslist-date').get_text())
    #            print(revisor[rownumber,0])
                revisor[rownumber,1] = j.find(class_='mw-userlink').get_text()
                byte_change_string = ''
                byte_change_string = j.find(class_='history-size').get_text()        
                byte_change_string = byte_change_string.replace('(','')
                byte_change_string = byte_change_string.replace(')','')
                if (byte_change_string.find('bytes') > -1):
                    byte_change_string = byte_change_string.replace(' bytes','')
                elif (byte_change_string.find('byte') > -1):
                    byte_change_string = byte_change_string.replace(' byte','')                    
                byte_change_string = byte_change_string.replace(',','')
    #        print(byte_change_string)
                if (byte_change_string == 'empty'):
                    revisor[rownumber,2] = 0
                else:
                    revisor[rownumber,2] = int(byte_change_string)
                plusorminusy = ''
                plusorminusy = j.find(class_='mw-plusminus-pos')
                if (plusorminusy == None):
                    plusorminusy = j.find(class_='mw-plusminus-neg')
                if (plusorminusy == None):
                    plusorminusy = j.find(class_='mw-plusminus-null')
                character_change_string = plusorminusy.get_text()
                character_change_string = character_change_string.replace('(','')
                character_change_string = character_change_string.replace(')','')
                character_change_string = character_change_string.replace('+','')
                character_change_string = character_change_string.replace(',','')
                revisor[rownumber,3] = int(character_change_string)
    #        revisor[rownumber,4] = name
    #        revisor[rownumber,5] = rownumber
#            print(rownumber," : ",revisor[rownumber])
            rownumber = rownumber + 1
        earliestrev =  datetime.datetime.strftime(np.min(revisor[:,0]),'%Y%m%d%H%M%S')
#        earliestrev = datetime.datetime.strftime(revisor[-1,0],'%Y%m%d%H%M%S')
#        print(len(revisor))
#        print(datetime.datetime.strftime(revisor[-1,0],'%Y-%m-%d'))
#        print(historyurl)
#    print(earliestrev)
    revisionarray[i,0] = (revisor.shape[0])
    revisionarray[i,1] = np.unique(revisor[:,[1]]).size
    print(i," : ",revisionarray[i])
#    print(name,": ","revisions: ",revisor.shape[0], " editors: ",np.unique(revisor[:,[1]]).size)
deadpeople = np.hstack((deadpeople,revisionarray))
#print(deadpeople)
print("Done!")

S. R. Nathan
0  :  [948 515]
Sabam Siagian
1  :  [19  9]
Sabri Yirmibeşoğlu
2  :  [47 18]
Sagan Lewis
3  :  [23  9]
Sahabzada Yaqub Khan
4  :  [268 152]
Sai Prashanth
5  :  [100  67]
Said Zahari
6  :  [66 41]
Saji Paravoor
7  :  [22 14]
Sally Beauman
8  :  [98 50]
Sally Brampton
9  :  [82 28]
Sally Katary
10  :  [34 10]
Salman Natour
11  :  [10  7]
Salvador Q. Quizon
12  :  [37 18]
Sam Bawlf
13  :  [38 10]
Sam Beall
14  :  [19 10]
Sam Beaver King
15  :  [39 15]
Sam Hulbert
16  :  [45 35]
Sam Spence
17  :  [170  98]
Samane Viyaket
18  :  [33 23]
Samantha Schubert
19  :  [27 14]
Samid Imanov
20  :  [16  7]
Samir Roychoudhury
21  :  [156  71]
Sammy Ellis
22  :  [104  50]
Samuel L. Green Jr.
23  :  [81 44]
Samuel Mumbengegwi
24  :  [44 26]
Samuel Odulana Odungade I
25  :  [45 15]
Samuel Rappaport
26  :  [18 11]
Samuel Robin Spark
27  :  [120  30]
Samuel Willenberg
28  :  [99 43]
Sándor Csjef
29  :  [3 2]
Sándor Csoóri
30  :  [77 48]
Sándor Tarics
31  :  [69 51]
Sandy McDonald
32  :  [144  

In [4]:
linkarray = np.zeros([len(deadpeople),1],dtype=int)
for i in range(0,len(deadpeople)):
#linkarray = np.zeros([1,1],dtype=int)
#for i in range(0,1):
    name = str(deadpeople[i,0])
#    name = "Miguel Gutierrez (Mexican footballer)"
#    name = "David Bowie"
#    name = "Agha Nasir"
#    name = "Marvin Minsky"
#    name = "Odo Fusi Pecci"
    name2 = urllib.parse.quote(name.replace(" ","_"))
    print(i,":",name)
    
    iteration = 0
    completed = 0
    nextpagelink = ''
    rownumber = 0
    while (completed == 0):
        iteration = iteration + 1
#        print('top of while loop.')
#        print("iteration: ",iteration)
#        print(rownumber)
        if (nextpagelink == ''):
            inbound_link_url = "https://en.wikipedia.org/w/index.php?title=Special:WhatLinksHere/"
            inbound_link_url += name2
            inbound_link_url += "&limit=500"
        else:
            inbound_link_url = nextpagelink
#        print("inbound_link_url = ",inbound_link_url)
        try:page = urllib.request.urlopen(inbound_link_url).read()
        except urllib.error.URLError as e:
                print("Epic fail reason: ",e.reason)
                linkarray[i,0] = 0 
                break
#        page = urllib.request.urlopen(inbound_link_url).read()
#        print("page = ",page)
        soup = bs(page, "html.parser")
        
        nextpage_element = soup.find('a',string="next 500")
        if (nextpage_element != None):
            nextpagelink = "https://en.wikipedia.org/" + nextpage_element.get('href')
#            print("nextpagelink = ",nextpagelink)
        else:
            completed = 1

        listoflinks = soup.find('ul',id='mw-whatlinkshere-list')

        inlinks = listoflinks.find_all('li')
        
        
        if (iteration == 1):
            linktracker = np.empty([len(inlinks),1],dtype=object)    
#            print('linktracker initialized to length ',len(linktracker))
        else:
#            print("size of linktracker before = ",linktracker.shape)
            moreroom = np.empty([len(inlinks),1],dtype=object)
#            print("size of moreroom = ",moreroom.shape)
            linktracker = np.vstack((linktracker,moreroom))
#            print("size of linktracker = ",linktracker.shape)
        
        for j in inlinks:
#            print(name)
#            print(rownumber)
#            print(j)
            linktracker[rownumber,0] = (bs.find(j,'a').get('href'))
            rownumber = rownumber + 1
        
#        print(linktracker)
    linkarray[i,0] = (linktracker.shape[0])
    print(linkarray[i,0])

deadpeople = np.hstack((deadpeople,linkarray))
print(deadpeople)

0 : S. R. Nathan
335
1 : Sabam Siagian
10
2 : Sabri Yirmibeşoğlu
14
3 : Sagan Lewis
13
4 : Sahabzada Yaqub Khan
144
5 : Sai Prashanth
15
6 : Said Zahari
18
7 : Saji Paravoor
5
8 : Sally Beauman
45
9 : Sally Brampton
32
10 : Sally Katary
5
11 : Salman Natour
9
12 : Salvador Q. Quizon
7
13 : Sam Bawlf
10
14 : Sam Beall
8
15 : Sam Beaver King
12
16 : Sam Hulbert
16
17 : Sam Spence
33
18 : Samane Viyaket
19
19 : Samantha Schubert
11
20 : Samid Imanov
4
21 : Samir Roychoudhury
62
22 : Sammy Ellis
185
23 : Samuel L. Green Jr.
160
24 : Samuel Mumbengegwi
59
25 : Samuel Odulana Odungade I
9
26 : Samuel Rappaport
12
27 : Samuel Robin Spark
12
28 : Samuel Willenberg
86
29 : Sándor Csjef
5
30 : Sándor Csoóri
256
31 : Sándor Tarics
19
32 : Sandy McDonald
44
33 : Sandy Pearlman
698
34 : Santiago J. Erevia
11
35 : Sarah Corp
9
36 : Sarah D. Grant
8
37 : Sarah Tait
40
38 : Sarath de Abrew
24
39 : Sascha Lewandowski
67
40 : Sasi Shanker
34
41 : Saulius Sondeckis
90
42 : Sayed Hussein Anwari
40
43 : Sa

In [5]:
#print("Column names: ",deadpeople.dtype.names)
#print("Fields: ",deadpeople.dtype.names)
#print(deadpeople)


#np.savetxt(outfilename, deadpeople, delimiter=",",fmt="%s,%i,%i,%i,%i")
#SciServer.SciDrive.upload(outfilename,deadpeople.all(),token=token)
#deadpeople
#import sys
#sys.setdefaultencoding('utf8')

#f = open('test2.csv','w')
#for i in range(0,len(deadpeople)):
#    unicodename = str(deadpeople[i,0]).encode('utf8')
#    f.write(str(unicodename))    
#    for j in range(1,4):
#        f.write(',')
#        f.write(str(deadpeople[i,j]))
#    f.write('\n')
#f.close()

#print(deadpeople)

x = pandas.DataFrame(deadpeople,columns=['name','article_length','versions','editors','inbound_links'])#,encoding="utf-8")
x.index.name = 'id'
outfile = str(os.getcwd())+'/deathout5.csv'
x.to_csv(outfile,encoding="utf-8")
print('Done!')

Done!
